In [6]:
from bayes_opt import BayesianOptimization
from keras.layers import *
import tensorboard
import sklearn.preprocessing
import tensorflow as tf
from tensorflow import keras
from packaging import version
from datetime import datetime
import pandas as pd
import numpy as np
import os
import glob
from linkmethods import get_pageviews, get_target, get_backlink_views

In [7]:
df_meisen= get_target("Meisen")
flatten_views = get_backlink_views("Meisen", get_target("Meisen"))
data = np.array(flatten_views).reshape((162,2558))[:,-250:-1]
print(data)
data.shape

            views
timestamp        
2015070100    2.0
2015070200    0.0
2015070300    4.0
2015070400    4.0
2015070500    1.0
...           ...
2022062700    1.0
2022062800    3.0
2022062900    1.0
2022063000    0.0
2022070100    3.0

[2558 rows x 1 columns]
            views
timestamp        
2015070100    7.0
2015070200   10.0
2015070300    6.0
2015070400    7.0
2015070500   11.0
...           ...
2022062700    5.0
2022062800    9.0
2022062900   11.0
2022063000   14.0
2022070100    9.0

[2558 rows x 1 columns]
            views
timestamp        
2015070100   1678
2015070200   1719
2015070300   1764
2015070400   1876
2015070500   2426
...           ...
2022062700   2452
2022062800   2906
2022062900   2974
2022063000   2838
2022070100   1921

[2558 rows x 1 columns]
            views
timestamp        
2015070100    0.0
2015070200    0.0
2015070300    0.0
2015070400    0.0
2015070500    0.0
...           ...
2022062700    0.0
2022062800    0.0
2022062900    2.0
2022063000    1.0
2022070

(162, 249)

In [8]:
target = np.array(df_meisen.views)[-250:-1]
target = np.array([np.copy(target) for _ in range((data.shape[0]))])
print(target)
target.shape

[[275 281 269 ... 192 190 180]
 [275 281 269 ... 192 190 180]
 [275 281 269 ... 192 190 180]
 ...
 [275 281 269 ... 192 190 180]
 [275 281 269 ... 192 190 180]
 [275 281 269 ... 192 190 180]]


(162, 249)

In [9]:
# Define the model.
model = keras.Sequential()

# 1 dense hidden layer with 250 nodes ~90+ acc
model.add(Input(shape=249)) # Input tensor
model.add(Dense(units=249)) # hidden layer 1,


model.compile(
    optimizer='Adam',
    loss='mean_absolute_percentage_error',
    metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 249)               62250     
                                                                 
Total params: 62,250
Trainable params: 62,250
Non-trainable params: 0
_________________________________________________________________


In [10]:
from sklearn.model_selection import train_test_split

# Separate the test data
x, x_test, y, y_test = train_test_split(data, target, test_size=0.15, shuffle=True)

# Split the remaining data to train and validation
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15, shuffle=True)

In [11]:
def get_model():
    # Define the model.
    model = keras.Sequential()

    # 1 dense hidden layer with 250 nodes ~90+ acc
    model.add(Input(shape=249)) # Input tensor
    model.add(Dense(units=249)) # hidden layer 1,

    return model

In [12]:
def fit_with(optimizers:str,loss:str):

    # Create the model using a specified hyperparameters.
    model = get_model()

    # Train the model for a specified number of epochs.

    model.compile(
    optimizer=optimizers,
    loss=loss,
    metrics=['accuracy'])

    # Train the model with the train dataset.
    history = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=100,
    validation_data=(x_val, y_val))

    # Return the accuracy.
    return min(history.history["accuracy"])

from functools import partial

verbose = 1
fit_with_partial = partial(fit_with,verbose)

In [13]:
# Bounded region of parameter space
pbounds = {'optimizers': ("SGD","RMSprop","Adam","AdamW","Adadelta","Adagrad","Adamax","Adafactor","Nadam","Ftrl"),
           'loss': ("binary_crossentropy","categorical_crossentropy","sparse_categorical_crossentropy","poisson","kl_divergence","mean_squared_error","mean_absolute_error","mean_absolute_percentage_error","mean_squared_logarithmic_error","cosine_similarity","huber_loss","log_cosh")}

optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(init_points=5, n_iter=20)


for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))
print(optimizer.max)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.